### Episódio 3
# Warm-up examples

In [ ]:
# imports e "configuração"
%matplotlib inline
from stimator import read_model
from ipywidgets import interact
import seaborn as sns

from classhelper import tc2df, last_time_as_series

styles = ['seaborn-whitegrid', 'seaborn-poster']

## exemplo 1: Sistema fechado, uma reação reversível Michaelis-Menten

![](images/closed.png)

#### O que se pode prever sobre este sistema?

- ????
-?????

In [ ]:
model1 = read_model("""
title Closed system, reversible Michaelis-Menten enzyme

v1: A -> P, (V * (A - P / Keq)) / (1 + A/KmA + P/KmP)

V = 1
Keq = 2

KmA = 2
KmP = 1

init: A = 12, P = 0

""")

In [ ]:
result = model1.solve(tf=40)
tc2df(result)

In [ ]:
model1.solve(tf=40).plot(fig_size=(12,8), style=styles)

### Changing parameters

- the initial value of A 
- the value of KmA

In [ ]:
@interact(A=(6, 12, 0.5), KmA=(1, 5, 0.1))
def change_A(A=12, KmA=2):
    model1.init.A = A
    model1.parameters.KmA = KmA
    model1.solve(tf=40).plot(fig_size=(12,8), yrange=(0,12), style=styles)

## Exemplo 2: sistema fechado, várias reações reversíveis Michaelis-Menten

![](images/closed_multi.png)

O que se pode prever sobre este sistema?

- ??????
- ????????

In [ ]:
model2 = read_model("""
title Linear pathway

v1: A -> B, (V * (A - B / Keq)) / (1 + A/KmA + B/KmB), V = 1, Keq = 2, KmA = 2, KmB = 1
v2: B -> C, (V * (B - C / Keq)) / (1 + B/KmB + C/KmC), V = 0.5, Keq = 2, KmB = 2, KmC = 1
v3: C -> D, (V * (C - D / Keq)) / (1 + C/KmC + D/KmD), V = 1, Keq = 2, KmC = 2, KmD = 1

init: A = 12

""")

In [ ]:
model2.solve(tf=150).plot(fig_size=(12,8), style=styles)

In [ ]:
steady_state = last_time_as_series(model2.solve(tf=2000))

steady_state

In [ ]:
A, B, C, D = steady_state.values

print(f'B/A = {B/A:.2f}')
print(f'C/B = {C/B:.2f}')
print(f'D/C = {D/C:.2f}')

#### Este sistema, por ser fechado, tende para um equilíbrio

## Exemplo 3: Sistema aberto
## várias reações reversíveis Michaelis-Menten
## (via metabólica genérica)

![](images/open_linear.png)

O que se pode prever sobre este sistema?

- ??????
- ????????

In [ ]:
model3 = read_model("""
title Linear pathway

v1: A -> B, (V * (A - B / Keq)) / (1 + A/KmA + B/KmB), V = 1, Keq = 2, KmA = 2, KmB = 1
v2: B -> C, (V * (B - C / Keq)) / (1 + B/KmB + C/KmC), V = 0.5, Keq = 2, KmB = 2, KmC = 1
v3: C -> D, (V * (C - D / Keq)) / (1 + C/KmC + D/KmD), V = 1, Keq = 2, KmC = 2, KmD = 1

vAin  : -> A, kin, kin = 1
vAout : A ->, koutA * A, koutA = 1

vout  : D ->, kout * D, kout = 2
""")

In [ ]:
model3.solve(tf=40).plot(fig_size=(12,8), style=styles)

#### Calcular os valôres de **estado estacionário**

In [ ]:
# Se simularmos até um tf muito elevado, estamos no estado estacionário

steady_state = last_time_as_series(model3.solve(tf=2000))

steady_state

In [ ]:
A, B, C, D = steady_state.values

print(f'B/A = {B/A:.2f}')
print(f'C/B = {C/B:.2f}')
print(f'D/C = {D/C:.2f}')

#### Os valores de estado estacionário **NÃO** são os de equilíbrio

#### Calcular os valôres das velocidades em **estado estacionário**

In [ ]:
steady_state = model3.solve(tf=20, outputs='>>').plot(fig_size=(12,8), style=styles)

#### Changing the parameter $kin$ (rate of input of A into the pathway)

In [ ]:
@interact(kin=(1, 5, 0.2))
def change_kin(kin=1):
    model3.parameters.vAin.kin = kin
    model3.solve(tf=20).plot(fig_size=(12,8), yrange=(0,4), style=styles)

#### Changing the parameter $kin$ (rate of input of A into the pathway), but displying the rates of reactions

In [ ]:
@interact(kin=(1, 5, 0.2))
def change_kin(kin=1):
    model3.parameters.vAin.kin = kin
    model3.solve(tf=20, outputs='v1 v2 v3'.split()).plot(fig_size=(12,8), yrange=(0,1), style=styles)